In [25]:
#dependencies
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime
import gmaps.datasets
from scipy.stats import linregress
from scipy import stats
import os

In [26]:
home_path = os.path.join ("../../GB/Resources/All_Hill_Prop_Value2019.csv")
home_value = pd.read_csv(home_path)
crime_path = os.path.join ("../Resources/combined_crime_data.csv")
crime_data = pd.read_csv(crime_path)

In [27]:
home_value.head()

,Unnamed: 0,SEQ_NO,PARCEL_ID,PropertyValue,NBRHD_CD,CENSUS_BK,PropAddress,City,ZipCode,BldgSqFt,YearBuilt
0,5,6,172701001000000000011U,462229,21100700,1.205700e+11,19859 ANGEL LN,ODESSA,33556,3972,1996
1,6,7,172701001000000000012U,496434,21100700,1.205700e+11,7320 W COUNTY LINE RD,ODESSA,33556,4643,2003
2,7,8,172701001000000000013U,358083,21100700,1.205700e+11,7212 W COUNTY LINE RD,ODESSA,33556,3269,1999
3,8,9,172701001000000000014U,819704,21100700,1.205700e+11,7222 W COUNTY LINE RD,ODESSA,33556,8203,2008
4,9,10,172701001000000000015U,325923,21100700,1.205700e+11,7326 W COUNTY LINE RD,ODESSA,33556,2608,2003


In [28]:
# create dataframe with properties less than 2500 sqft
home_value_19 = home_value.loc[(home_value["BldgSqFt"] <= 2500)]
home_value_19.head()

,Unnamed: 0,SEQ_NO,PARCEL_ID,PropertyValue,NBRHD_CD,CENSUS_BK,PropAddress,City,ZipCode,BldgSqFt,YearBuilt
10,15,16,17270100100000000001BU,317202,21100700,1.205700e+11,7214 COLLEY RD,ODESSA,33556,2068,1986
12,17,18,172701001000000000020U,161380,21100700,1.205700e+11,19901 ANGEL LN,ODESSA,33556,1728,1973
13,18,19,172701001000000000021U,345106,21100700,1.205700e+11,19913 ANGEL LN,ODESSA,33556,1924,1976
15,22,23,172701001000000000025U,291198,21100700,1.205700e+11,7502 W COUNTY LINE RD,ODESSA,33556,2482,2003
18,25,26,172701001000000000028U,292881,21100700,1.205700e+11,7340 COLLEY RD,ODESSA,33556,2274,1982


In [29]:
# Drop outlier indices
home_values_clean = home_value_19

home_values_clean["ZipCode"]
# Calculate the average property value by ZIP
home_average = home_value.groupby(['ZipCode'])

home_average_df = pd.DataFrame(home_average["PropertyValue"].mean())
home_average_df = home_average_df.reset_index()

home_average_df.head()

,ZipCode,PropertyValue
0,33503,381777.000000
1,33510,232671.184950
2,33511,326118.147603
3,33527,274593.132697
4,33534,222944.543907


In [38]:
crime_data.head()

,Reported Date,Case Number,Crime Type,Crime Type Extension,year,Address,zip
0,1/1/2016,2016-281,DUI,DUI (DRIVING UNDER INFLUENCE),2016,FLORIDA AV N / WINDWOOD OAKS DR,33613
1,1/1/2016,2016-60,BATTERY,BATTERY-SIMPLE,2016,13600 BLOCK GRAGSTON CR,33613
2,1/2/2016,2016-2990,BURGLARY,THEFT FROM A VEHICLE,2016,4000 BLOCK DREAM OAK PL,33613
3,1/2/2016,2016-3345,THEFT,THEFT FROM A BUILDING,2016,13700 BLOCK 42ND ST N,33613
4,1/2/2016,2016-3552,BATTERY,BATTERY-SIMPLE,2016,14200 BLOCK NEBRASKA AV N,33613


In [43]:
# Add average property value to crime dataframe
for i, row in home_average_df.iterrows():
    crime_data.loc[crime_data['zip'] == row['ZipCode'], "Avg Prop Value"] = row['PropertyValue']

crime_data.head()

,Reported Date,Case Number,Crime Type,Crime Type Extension,year,Address,zip,Avg Prop Value
0,1/1/2016,2016-281,DUI,DUI (DRIVING UNDER INFLUENCE),2016,FLORIDA AV N / WINDWOOD OAKS DR,33613,429963.556743
1,1/1/2016,2016-60,BATTERY,BATTERY-SIMPLE,2016,13600 BLOCK GRAGSTON CR,33613,429963.556743
2,1/2/2016,2016-2990,BURGLARY,THEFT FROM A VEHICLE,2016,4000 BLOCK DREAM OAK PL,33613,429963.556743
3,1/2/2016,2016-3345,THEFT,THEFT FROM A BUILDING,2016,13700 BLOCK 42ND ST N,33613,429963.556743
4,1/2/2016,2016-3552,BATTERY,BATTERY-SIMPLE,2016,14200 BLOCK NEBRASKA AV N,33613,429963.556743


In [44]:
index_crime_19 = crime_data[(crime_data['year'] != 2019)].index 
crime_data_19 = crime_data.drop(index_crime_19, inplace = False)

crime_data_19.head()

,Reported Date,Case Number,Crime Type,Crime Type Extension,year,Address,zip,Avg Prop Value
6687,1/1/2019,2019-1374,BATTERY,BATTERY-SIMPLE,2019,1100 BLOCK 140TH AV E,33613,429963.556743
6688,1/1/2019,2019-1374,DRUGS,DRUGS/NARCOTICS,2019,1100 BLOCK 140TH AV E,33613,429963.556743
6689,1/1/2019,2019-1374,DRUGS,DRUG PARAPHERNALIA,2019,1100 BLOCK 140TH AV E,33613,429963.556743
6690,1/1/2019,2019-1202,BURGLARY,BURGLARY RESIDENCE / NO FORCE,2019,3600 BLOCK PALM CROSSING DR,33613,429963.556743
6691,1/1/2019,2019-1342,BURGLARY,BURGLARY RESIDENCE / NO FORCE,2019,3600 BLOCK PALM CROSSING DR,33613,429963.556743


In [36]:
#Create scatter plot w/ regression line for school grade vs. prop value

# Set x and y values for scatter plot
x_values = crime_data_19['Avg Prop Value']
y_values = crime_data_19['Crime Type']
# y_values = crime_data_19['Reported Date']
# y_values = crime_data_19['Case Number']
# y_values = crime_data_19['year']
# y_values = crime_data_19['zip']

# Perform a linear regression on school grade vs. average property value
(slope, intercept, rvalue, pvalue, stderr) = linregress(x_values, y_values)

# Get regression values
regress_values = x_values * slope + intercept

# Create line equation string
line_eq = "y = " + str(round(slope,2)) + "x +" + str(round(intercept,2))

# Create scatter plot and
# set plot specs
plt.scatter(x_values, y_values)
plt.plot(x_values,regress_values,"r-")
plt.xlabel('Average Property Value in School ZIP (by $)')
plt.ylabel("School Grade (% of total possible points)")
plt.ylim(0, 100)
plt.xticks(rotation=45)
plt.annotate(line_eq,(5,-40),fontsize=15,color="red")
plt.grid(which='major', axis='both')
plt.title("School Grade vs. Average Property Value")

# Print r value
print(f"The r- is: {rvalue**2}")

# Display plot
plt.show()

KeyError: 'Avg Prop Value'